In [2]:
# Imports
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score
from torchvision import models
from torch.utils.data import DataLoader, Dataset
import joblib

In [ ]:
# Step 1: Load and Preprocess the Dataset
class FERDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        label = int(row['emotion'])
        image = np.array(row['pixels'].split(), dtype=np.uint8).reshape(48, 48)  # FER2013 images are 48x48
        image = torch.tensor(image, dtype=torch.float32).unsqueeze(0)  # Add channel dimension
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Load FER2013 data
df = pd.read_csv('fer20131.csv')

# Split into training and testing datasets
train_data, test_data = train_test_split(df, test_size=0.2, stratify=df['emotion'], random_state=42)

# Create PyTorch datasets
train_dataset = FERDataset(train_data)
test_dataset = FERDataset(test_data)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)